## pdftree
根节点为virtualNode，根据headline及其level构建pdftree。
没有headline的默认为根节点的子节点

In [28]:
from pdfextractor import extractor
import pdfextractor
import pickle
import jieba
import pandas as pd
import re

In [ ]:
d = pd.read_csv('/home/zhuyuhe/mydata/jmd/percent/jmd_text_map.txt', header = None, sep = '\t', encoding = 'utf-8')
d.columns = ['index','文本', '意图', 'pdfurl', 'pdf文本']
d = d[d['文本'].apply(lambda x: x.__len__() > 10)]
d = d[d['文本'].duplicated() == False]
d.to_csv('/home/zhuyuhe/mydata/jmd/percent/jmd_text_map.txt', header = None,index=None, sep = '\t', encoding = 'utf-8')

In [ ]:
a = "创显将大力发展软硬件结合的教学解决方案，配套以不同层次的专业培训服务"
pdflist = ['2017年，公司将大力发展软硬件结合的视频教学解决方案，形成精品录播室，标准录播，配套以不同的专业培训服务', '2017年，公司将在既有的教师化信息能力平台业务基础上，进一步深化细化培养服务']
print(isMatch(a, pdflist, 0.6))

In [2]:
pdfurl = ''
savePath = '/home/zhuyuhe/mydata/jmd/percent/'
# fileObject = open(savePath + 'jmd_pdf_map.txt', 'w') 
def write_unicode(text, charset='utf-8'):
    return text.encode(charset)
def MatchTwoText(t1, t2, threshold = 0.5):
    """
    检查两个句子是否匹配
    """
#     print t1
    ct1 = ' '.join(jieba.cut(t1)).split()
    ct2 = ' '.join(jieba.cut(t2)).split()
    commonL = set(ct1).intersection(set(ct2)).__len__()
#     print commonL
#     print ct1
#     print set(ct1).__len__()
    if commonL * 1.0 / set(ct1).__len__() > threshold:
#         print("match " + str(commonL * 1.0/ set(ct1).__len__() * 100) + '% ')
#         print("文章句子:" + '\n' + t1)
#         print("PDF句子: " + '\n' + t2)
#         print('\n')
#         fileObject.write(t1.encode('utf-8'))
#         fileObject.write('\t')
#         fileObject.write(t2.encode('utf-8'))
#         fileObject.write('\t')
#         fileObject.write(pdfurl)
#         fileObject.write('\n')
        return True
    return False
    
def isMatch(text, pdfTextList, threshold = 0.5):
    """
    @parameter text: 待检查句子
    @parameter pdfTextList: PDF句子与表格信息列表
    @parameter threshold: 相似度为多少时返回true
    @return： boolean值，pdf列表中是否有与待检查句子非常相似的句子
    """
    for pdftext in pdfTextList:
        if MatchTwoText(text, pdftext, threshold):
            return True
    return False

def get_node_texts(pdf_nodes):
    '''Given a list of nodes, get the texts of these nodes
    '''
    texts = []
    for node in pdf_nodes:
        text = node.obj.get_text()
        if len(text) <= 25:
            continue
        if u'□' in text or u'√' in text:
            continue
        texts.append(text)
    return texts

#percent: 取pdf的前多少页
def extractPDF(pdfPath, percent, source = ['table', 'text', 'para'], headline = None):
    savePath = '/home/zhuyuhe/mydata/jmd/percent/'
    ex = extractor.StockFinanceExtractor(pdfPath, percent)

    nodes = ex.get_pdf_objects(filter_headline=True)

    paraList = []
    textList = []
    textNodes = []
    tableList = []
    tableNodes = []
    senList = []
    for node in nodes:
        if isinstance(node.obj, pdfextractor.pdftree.VirtualNode):
            continue
        if isinstance(node.obj, pdfextractor.page.Table):
            tableNodes.append(node)
        elif  not isinstance(node.obj, pdfextractor.page.Catalog):
            textNodes.append(node)
    if 'table' in source:
        tableList = get_node_texts(tableNodes)
        tableList = [t.replace('|', '').split('\n') for t in tableList]
        #二维数组转化为一维
        tableList = sum(tableList,[])
    textList = get_node_texts(textNodes)
    if 'para' in source:
        paraList = [t.replace('\n', '') for t in textList]
        paraList = filter(lambda x: x != '', paraList)
    elif 'text' in source:
        senList = sum([t.encode('utf-8').split('。') for t in textList],[])
        senList = [t.replace('\n', '') for t in senList]
        senList = filter(lambda x: x != '', senList)

    pdf = []
#     pdf.append(textList)
#     pdf.append(tableList)
#     pdf = sum(pdf, [])
    pdf.extend(senList)
    pdf.extend(tableList)
    pdf.extend(paraList)
    print pdf.__len__()
    
    return pdf


#     
#     fileObject = open(savePath + 'cxkjTextList.txt', 'w')  
#     for text in textList:  
#         fileObject.write(text)  
#         fileObject.write('\n')
#     fileObject.close()
#     fileObject2 = open(savePath + 'cxkjTableList.txt', 'w')  
#     for table in tableList:  
#         fileObject2.write(write_unicode(table))  
#         fileObject2.write('\n')  
#     fileObject2.close()

In [17]:
print MatchTwoText('报告显示，方直科技在一季度实现营业收入34,743,435.23元，由于教育系统征订销售收入和其他产品销售收入的增加，营收同比增长5.48%。',
                   '公司在2016年第一季度实现营业收入34,743,435.23元 ，营业收入较上年同期增长5.48%，主要来自于教育系统征订销售收入增加和其他产品销售收入增加。公司核心产品业务及其结构未发生重大变化，金太阳教育软件的开发、销售业务仍是公司的核心业务。华南、华北和华东三个地区是公司产品销售比较集中的区域，报告期内各地区销售收入占主营业务收入比例分别为 70.48%、16.55%、7.2%。')

True


In [4]:
#根据pdfurl和文章内容，找出每篇文章有多大比例来自pdf。并将pdf的段落保存下来
import sys
import pickle
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

seasonArticle = ['http://disclosure.szse.cn/finalpage/2016-10-25/1202780794.PDF', 'http://disclosure.szse.cn/finalpage/2016-04-26/1202239317.PDF', 'http://disclosure.szse.cn/finalpage/2016-10-27/1202793355.PDF','http://disclosure.szse.cn/finalpage/2017-04-26/1203386046.PDF']
yearlyArticle = [u'http://www.cninfo.com.cn/finalpage/2017-04-19/1203340942.PDF', 
                 u'http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF',
                 u'http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2017-04-15/601801_2016_n.pdf', 
                 u'http://disclosure.szse.cn/finalpage/2017-07-28/1203739610.PDF', 
                 u'http://www.cninfo.com.cn/finalpage/2017-06-20/1203635303.PDF', 
                 u'http://disclosure.szse.cn/finalpage/2017-08-10/1203778984.PDF', 
                 u'http://disclosure.szse.cn/finalpage/2017-03-28/1203210341.PDF', 
                 u'http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2016-08-25/600661_2016_z.pdf', 
                 u'http://www.cninfo.com.cn/finalpage/2017-07-28/1203751786.PDF', 
                 u'http://disclosure.szse.cn/finalpage/2017-04-25/1203379575.PDF', 
                 u'http://www.cninfo.com.cn/finalpage/2017-05-10/1203508904.PDF',
                 u'http://disclosure.szse.cn/finalpage/2017-04-24/1203360123.PDF',
                 u'http://www.cninfo.com.cn/finalpage/2017-04-27/1203451818.PDF', 
                 u'http://disclosure.szse.cn/finalpage/2017-03-18/1203174359.PDF',
                 u'http://www.cninfo.com.cn/finalpage/2017-04-19/1203341493.PDF', 
                 u'http://www.cninfo.com.cn/finalpage/2017-04-21/1203369775.PDF',
                 u'http://www.cninfo.com.cn/finalpage/2017-07-26/1203737887.PDF',
                 u'http://disclosure.szse.cn/finalpage/2017-03-21/1203181705.PDF',
                 u'http://disclosure.szse.cn/finalpage/2017-04-11/1203274908.PDF']

def splitByPeriod(P):
    """
    split paragraph to sentences
    """
    senList = sum([t.encode('utf-8').split('。') for t in P],[])
    senList = [t.replace('\n', '') for t in senList]
    senList = filter(lambda x: x != '', senList)
    return senList

savePath = '/home/zhuyuhe/mydata/jmd/percent/'
# fp = open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_mapping2.pkl', 'rb')
fp = open('/home/zhuyuhe/mydata/jmd/jmt_jmd_mappings2.pkl', 'rb')
jmdata_mapping = pickle.load(fp)

content_url_map = {}
for d in jmdata_mapping:
    content_url_map[d['pdf_url']] = d['content']

urls = content_url_map.keys()

num = 1
result = {}
jmdata_map_headline = {}

def parsePDF(urls,style, cond):  
    jmdata_map_headline[style] = {}
    for pdfurl in urls:
        print pdfurl
        jmdata_map_headline[style][pdfurl] = {}

        cxkj = content_url_map[pdfurl]
        #print cxkj
        cxkj = cxkj.strip().encode('utf-8').split('。')
        cxkj = [x.replace('\n','') for x in cxkj]

        try :
            pdflist = getPara(pdfurl, 0.5, cond)
        except BaseException:
            print 'error extractPDF'
            continue

        print "------"
        jmdata_map_headline[style][pdfurl]['content'] = cxkj
        jmdata_map_headline[style][pdfurl]['para'] = pdflist
        jmdata_map_headline[style][pdfurl]['sen'] = splitByPeriod(pdflist)
        jmdata_map_headline[style][pdfurl]['mapcontent'] = []
#         for i in cxkj:
#             print i
#             print '-'
#         print '**********'
#         for i in pdflist:
#             print i
#             print '-'

    #     fileObject = open(savePath + 'cxkjTextList.txt')
    #     textlist = fileObject.read().split('\n')
    #     fileObject = open(savePath + 'cxkjTableList.txt')
    #     tablelist = fileObject.read().split('\n')
    #     fileObject.close()

    #     PDFTextList = []
    #     PDFTextList.append(textlist)
    #     PDFTextList.append(tablelist)
    #     PDFTextList = sum(PDFTextList, [])
    #     print PDFTextList.__len__()
        count = 0
        for text in cxkj:
            if not (len(text) == 0):
                if isMatch(text, pdflist, 0.6):
                    jmdata_map_headline[style][pdfurl]['mapcontent'].append(text)
                    count+=1

        print str(count * 1.0/ cxkj.__len__() * 100) + '% 的句子来自PDF'
        result[pdfurl] = count * 1.0/ cxkj.__len__() * 100
        print('***************************')
    
# fileObject.close()

parsePDF(seasonArticle, 'season', ['重大风险提示', '业务回顾和展望'])
parsePDF(yearlyArticle, 'year', ['讨论与分析', '公司业务概要'])

#parsePDF(['http://disclosure.szse.cn/finalpage/2016-04-26/1202239317.PDF'], 'season', ['重大风险提示', '业务回顾和展望'])


http://disclosure.szse.cn/finalpage/2016-10-25/1202780794.PDF
Download pdf time: 1s
download http status code: 200
Pdfminer Parsed time: 4s


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


Construct Pdf Tree time: 0s
------


Loading model cost 0.374 seconds.
Prefix dict has been built succesfully.


64.2857142857% 的句子来自PDF
***************************
http://disclosure.szse.cn/finalpage/2016-04-26/1202239317.PDF
Download pdf time: 0s
download http status code: 200
Pdfminer Parsed time: 7s
Construct Pdf Tree time: 0s
------
63.6363636364% 的句子来自PDF
***************************
http://disclosure.szse.cn/finalpage/2016-10-27/1202793355.PDF
Download pdf time: 1s
download http status code: 200
Pdfminer Parsed time: 10s
Construct Pdf Tree time: 0s
------
28.5714285714% 的句子来自PDF
***************************
http://disclosure.szse.cn/finalpage/2017-04-26/1203386046.PDF
Download pdf time: 1s
download http status code: 200
Pdfminer Parsed time: 5s
Construct Pdf Tree time: 0s
------
75.0% 的句子来自PDF
***************************
http://www.cninfo.com.cn/finalpage/2017-04-19/1203340942.PDF
Download pdf time: 2s
download http status code: 200
Pdfminer Parsed time: 20s
Construct Pdf Tree time: 0s
------
75.0% 的句子来自PDF
***************************
http://disclosure.szse.cn/finalpage/2017-04-21/1203344261

In [7]:
k = seasonArticle[0]
for i in jmdata_map_headline['season'][k]['content']:
    print i
    print '--'

今日傍晚， 方直科技 （300235）发布了2016年第三季度报告，报告显示，2016年第三季度，其营业总收入为3610.01万元，归属于上市公司股东的净利润为1331.05万元
--
 营收数据 统计数据显示，截至报告期末，方直科技总资产为3.80亿元
--
其第三季度营业总收入为3610.01万元，同比下降14.40%；归属于上市公司股东的净利润为1331.05万元，同比增加5.20%
--
 2016年前三季度，方直科技营业总收入为7936.14万元，同比下降2.89%；归属于上市公司股东的净利润为1817.50万元，同比下降22.32% 据悉，本报告期末应收账款比年初增长108.12%，主要原因是秋季教育系统征订业务确认收入形成的应收账款较大所致
--
 业务展望 前三季度，方直科技核心产品业务及其结构未发生重大变化，金太阳教育软件的开发、销售业务仍是公司的核心业务
--
华南、华北和华东三个地区是产品销售比较集中的区域，报告期内各地区销售收入占主营业务收入比例分别为67.08%、17.92%、8.6%
--
 未来，在产品研发方面，方直科技将继续巩固强化公司在内容资源方面的优势，建立和完善语文、英语、数学等多学科、多版本的同步教学优质资源开发
--
不断丰富金太阳产品集群多样性，加快推进各应用端全覆盖，促使更多的免费使用用户转化为购买用户
--
 另一方面，方直科技将不断加大对网络技术开发、学习资源开发及网络产品运营人力的投入，以加速向互联网企业转型
--
 风险应对 目前方直科技的销售区域仍主要以珠三角、长三角及京津地区等大中城市为主，市场区域相对集中并存在一定风险，可能对其未来盈利能力造成影响
--
 方直科技将通过技术研发和产品拓展计划、营销渠道拓展计划解决上述产品及销售区域较为集中的风险
--
 如果方直科技对行业关键技术的发展动态不能及时掌控，对教育教学当中的新理念以及广大教师和学生的教学需求不能正确把握，研发的产品将不具备竞争力
--
 因此，方直科技将积极投入同步教学软件及在线教育网络服务开发平台的开发，加强基础教育项目研究
--
 来源：                                    芥末堆
--


In [3]:
def getPara(pdfurl, percent = 0.5, cond = ['业务回顾和展望', '重大风险提示']):
    ex = extractor.StockFinanceExtractor(pdfurl, percent)
    nodes = ex.get_pdf_objects()
    headlines = getHeadlines(nodes, 5)
#     print 'headlines length : ' + str(len(headlines))
    targethl = []
    res = []
    for hl in headlines:
        for c in cond:
            if c in hl.obj.get_text():
                targethl.append(hl)
#     print 'targethl :' + str(len(targethl))
#     for t in targethl:
#         print t.obj.get_text()
    if len(targethl) == 0:
        paraList = []
        for node in nodes:
            if isinstance(node.obj, pdfextractor.paragraph.Paragraph):
                paraList.append(node.obj.get_text())
        return paraList

    for t in targethl:
        res.extend(get_para_recu(t))
    return res
    
def get_para_recu(node):
    res = []
    if isinstance(node.obj, pdfextractor.paragraph.Paragraph):
        return res
    child_headlines = get_child_headlines(node)
    child_paras = get_child_paragraphs(node)
    for c in child_paras:
        res.append(filter_text(c.obj.get_text()))
    for c in child_headlines:
        res.extend(get_para_recu(c))
    return res

def filter_text(text):
    if len(text) <= 25:
        return ''
    if u'□' in text or u'√' in text:
        return ''
    return text

    

    
    
def get_child_headlines(pdf_node):
    '''Get headlines of children of a node
    '''
    headlines = []
    for node in pdf_node.children:
        if isinstance(node.obj, pdfextractor.paragraph.Headline):
            headlines.append(node)
    return headlines

def get_child_paragraphs(pdf_node):
    '''Get paragraphs of children of a node
    '''
    paragraphs = []
    for node in pdf_node.children:
        if isinstance(node.obj, pdfextractor.paragraph.Paragraph):
            paragraphs.append(node)
    return paragraphs

def getHeadlines(nodes, level):
    """
    return headline-nodes from given nodes
    """
    re = []
    for node in nodes:
        if isinstance(node.obj, pdfextractor.paragraph.Headline) and node.obj.level <= level:
            re.append(node)
    return re

def getMaxLevel():
    pass
        
        
             

# target = getTargetNode()

# print get_child_paragraphs(get_child_headlines(get_child_headlines(target)[0])[0])[0]

In [11]:
print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['content'])
print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['mapcontent'])
print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['para'])
print len(jmdata_map['http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF']['sen'])
jmdata_map = dict(filter(lambda x: len(x[1]) != 0, jmdata_map.items()))

18
12
145
255


In [5]:
with open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_map_headline.pkl', 'wb') as pickle_file:
     pickle.dump(jmdata_map_headline, pickle_file , protocol=2)

In [12]:
with open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_map.pkl', 'wb') as pickle_file:
     pickle.dump(jmdata_map, pickle_file , protocol=2)

In [ ]:
pdftext = dataJoin[dataJoin['pdfurl'] == 'http://disclosure.szse.cn/finalpage/2016-10-27/1202793355.PDF'][dataJoin['pdf文本'] != '']['pdf文本']
pdftext.to_csv('/home/zhuyuhe/mydata/jmd/predict/pdftext.txt', header = None, index=None, sep = '\t', encoding = 'utf-8')

In [ ]:
#计算pdf意图符合文章意图的句子比例
pdftext = dataJoin[dataJoin['pdfurl'] == 'http://disclosure.szse.cn/finalpage/2016-10-27/1202793355.PDF'][dataJoin['pdf文本'] != '']
pdftext_intent = pd.read_csv('/home/zhuyuhe/mydata/jmd/predict/cxkjpdftext_predict.txt', header = None)
pdftext['pdfintent'] = pdftext_intent.values
pdftext['pdfintent'] = pdftext['pdfintent'].apply(lambda x: x.replace('__label__', ''))
pdftext['意图'] = pdftext['意图'].apply(lambda x: '|'.join(x.split('|')[:2]))
bol = pdftext['意图'] == pdftext['pdfintent']
print "文本句子有 " + str(len(bol)) + " 句"
print "pdf意图符合文章意图的句子比例为" + str(bol.value_counts()[1] * 1.0 / len(bol) * 100) + "%"

In [ ]:
print qtjypdftext[43]

In [ ]:
import numpy as np
result = np.array(result.values())
result[result != 0]
#0%的原因： 1.科大讯飞  -  表格描述；上一季度财报描述  ×2 
#           2.新南洋    -  财报中大部分均为表格，并且文章使用了其他季度的财报
#           3.中国高科  -  前两句话描述财报表格，
#           4.洪涛股份  -  文章与财报不匹配，2015年年报 = 2016年第一季度财报？

In [ ]:
text_nodes = []

In [ ]:
for n in nodes:
    if isinstance(n.obj, pdfextractor.pdftree.VirtualNode):
        continue
    if len(n.obj.get_text()) <= 25:
        continue
    else:
        text = n.obj.get_text()
        if u'□' in text or u'√' in text:
            continue
        text_nodes.append(n)
        print n

In [ ]:
node= text_nodes[10]

In [ ]:
isinstance(node.obj,pdfextractor.paragraph.Paragraph)

In [66]:
#抽出特定条件下（headline）的段落
ex = extractor.StockFinanceExtractor('http://www.cninfo.com.cn/finalpage/2017-04-29/1203425130.PDF', 0.5)

nodes = ex.get_pdf_objects()

Download pdf time: 0s
download http status code: 200
Pdfminer Parsed time: 3s
Construct Pdf Tree time: 0s


In [72]:
['zhu'+'yuhe', 'shenjiajia']

['zhuyuhe', 'shenjiajia']

In [71]:
print nodes[2].obj.get_text()

2017 年第一季度报告


In [12]:
def get_table_nodes(nodes):
    tables = []
    for node in nodes:
        if isinstance(node.obj, pdfextractor.table.Table):
            tables.append(node)
    return tables
def get_headline_nodes(nodes):
    headlines = []
    for node in nodes:
        if isinstance(node.obj, pdfextractor.paragraph.Headline):
            headlines.append(node)
    return headlines
def get_paragraph_headlines(pdf_node):
    '''Get headlines of a paragraph
    Input:
        - pdf_node: a pdf node
    Output:
        - headlines: a list of sorted headline nodes
        - texts: a list of sorted headline texts
    '''
    headlines = []
    node = pdf_node.parent
    while not isinstance(node.obj,pdfextractor.pdftree.VirtualNode):
        headlines.append(node)
        node = node.parent
    headlines = list(reversed(headlines))
    return headlines, get_node_texts(headlines)

In [67]:
tablenodes = get_table_nodes(nodes)
def get_table_column_name(node):
    if not isinstance(node.obj, pdfextractor.table.Table):
        raise TypeError("node is not a table object")
    return node.obj.get_text().split('\n')[0].split('|')
def get_table_info(node, match = None, col_cond = None):
        """
        return the table content match the given condition
        """
        content = []
        if not isinstance(node.obj, pdfextractor.table.Table):
            raise TypeError("node is not a table object")
        if match is None:
            return content
        if not (type(match) == list):
            raise TypeError("parameter 'match' should be list type")
        if (not col_cond is None) and (not col_cond in node.obj.get_text().split('\n')[0]):
            return content
        content.append(get_table_column_name(node))
        rows = node.obj.get_text().split('\n')[1:]
        for r in rows:
            for m in match:
                if m in r:
                    content.append(r.split('|'))
                    
        if len(content) == 1:
            return []
        return content
        
        
for t in tablenodes:
    if get_table_info(t, ['营业收入（元）', '归属于上市公司股东的净利润（元）']).__len__() != 0:
        print get_table_info(t, ['营业收入（元）', '归属于上市公司股东的净利润（元）'])
        
    for i in get_table_info(t, ['营业收入', '净利润'], col_cond = '变动原因'):
        print i

# for t in tablenodes:
#     print t.obj.get_text().split('\n')[0]

[[u' ', u'\u672c\u62a5\u544a\u671f ', u'\u4e0a\u5e74\u540c\u671f ', u'\u672c\u62a5\u544a\u671f\u6bd4\u4e0a\u5e74\u540c\u671f\u589e\u51cf'], [u'\u8425\u4e1a\u6536\u5165\uff08\u5143\uff09 ', u'4,412,184,212.16', u' 3,543,649,778.82', u' 24.51'], [u'\u5f52\u5c5e\u4e8e\u4e0a\u5e02\u516c\u53f8\u80a1\u4e1c\u7684\u51c0\u5229\u6da6\uff08\u5143\uff09 ', u'230,667,840.43', u' 321,923,477.66', u' -28.35']]


In [23]:
import numpy as np
table = np.array([s.split('|') for s in get_table_nodes(nodes)[0].obj.get_text().split('\n')])
table

array([[u' ', u'\u672c\u62a5\u544a\u671f ', u'\u4e0a\u5e74\u540c\u671f ',
        u'\u672c\u62a5\u544a\u671f\u6bd4\u4e0a\u5e74\u540c\u671f\u589e\u51cf '],
       [u'\u8425\u4e1a\u6536\u5165\uff08\u5143\uff09 ',
        u'1,692,627,343.39', u' 1,231,711,694.89', u' 37.42'],
       [ u'\u5f52\u5c5e\u4e8e\u4e0a\u5e02\u516c\u53f8\u80a1\u4e1c\u7684\u51c0\u5229\u6da6\uff08\u5143\uff09 ',
        u'101,646,352.08', u' 66,028,042.54', u' 53.94'],
       [ u'\u5f52\u5c5e\u4e8e\u4e0a\u5e02\u516c\u53f8\u80a1\u4e1c\u7684\u6263\u9664\u975e\u7ecf\u5e38\u6027\u635f\u76ca\u7684\u51c0\u5229\u6da6\uff08\u5143\uff09 ',
        u'94,316,430.2', u' 48,739,088.0', u' 93.51'],
       [ u'\u7ecf\u8425\u6d3b\u52a8\u4ea7\u751f\u7684\u73b0\u91d1\u6d41\u91cf\u51c0\u989d\uff08\u5143\uff09 ',
        u'82,959,884.21', u' -203,912,566.88', u' 140.68'],
       [u'\u57fa\u672c\u6bcf\u80a1\u6536\u76ca\uff08\u5143/\u80a1\uff09 ',
        u'0.03', u' 0.02', u' 50.00'],
       [u'\u7a00\u91ca\u6bcf\u80a1\u6536\u76ca\uff08

## 预测

In [ ]:
import requests

In [ ]:
host = 'https://alpha-surreal.aidigger.com/api/v1/classification/intent/finance/0'

In [ ]:
hd_intents = []
all_intents = []
for node in text_nodes:
    content = node.obj.get_text()
    headline = get_paragraph_headlines(node)[1][-1]
    intents = requests.post(host,json={"query":content}).json()
    head_intents = requests.post(host,json={"query":headline}).json()
    for i,item in enumerate(head_intents['predict']):
        intents['predict'][i]['prob'] += item['prob']
    all_intents.append(intents)
    intent = max(intents['predict'],key=lambda item:item['prob'])
    hd_intents.append(intent)

In [ ]:
def print_intent(it):
    for i,item in enumerate(it):
        print i,item['name'],item['prob']

In [ ]:
print_intent(all_intents[0]['predict'])

In [ ]:
for i,intent in enumerate(hd_intents):
    print i
    print text_nodes[i]
    print ' ==> '.join(get_paragraph_headlines(text_nodes[i])[1])
    print intent['name'],'\n'


# 根据芥末堆提供的意图模板，得到意图推荐的准确率和召回率

In [ ]:
query = requests.post("https://surreal.aidigger.com/api/v1/classification/intent/finance/0",json={ 
    "query": "公司立足校园、面向家庭提供教育信息服务，深耕教育十余年已形成覆盖校内外多场景、涵盖教育主管部门、学校、教师、家长及学生等 B 端、C 端主体的业务服务体系。公司针对不同用户群体多场景、多元化需求，持续优化服务供给，逐步构建了家校互动升级业务、EdSaaS 业务、学科升学业务及继续教育业务，报告期内公司主营业务未发生变化。"
}).json()

In [ ]:
def hasCN(s):
    """
    判断一段文本是否有中文
    """
    for ch in s.decode('utf-8'):
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False



In [ ]:
dataJoin = pd.read_csv('/home/zhuyuhe/mydata/jmd/percent/dataJoin.txt', delimiter='\t', header = None)

dataJoin.columns = ['文本', '意图', 'pdfurl']

dataJoin = dataJoin[dataJoin['文本'].apply(lambda x: len(x) > 30)]
dataJoin = dataJoin[dataJoin['文本'].duplicated() == False]
dataJoin = dataJoin[dataJoin['文本'].apply(lambda x: hasCN(x))]

dataJoin.index = range(len(dataJoin))
dataJoin['意图'] = dataJoin['意图'].apply(lambda x: intentMap(x))

dataJoin = dataJoin[dataJoin['意图'].apply(lambda x: x!= 'Other')]

In [ ]:
for i in dataJoin['意图'].unique():
    print i

In [ ]:
a = dataJoin[dataJoin.pdfurl == 'http://disclosure.szse.cn/finalpage/2016-10-27/1202791188.PDF']
b = dataJoin[dataJoin.pdfurl == 'http://disclosure.szse.cn/finalpage/2016-10-25/1202780960.PDF'] 
c = dataJoin[dataJoin.pdfurl == 'http://www.cninfo.com.cn/finalpage/2017-04-27/1203451818.PDF']
print a.loc[119, '文本']

In [ ]:
len(dataJoin)

In [ ]:
print a['意图'].value_counts()
print b['意图'].value_counts()
print c['意图'].value_counts()

In [36]:
import operator
d = {'a':1, 'b':2, 'c':3}
sorted(d.items(), key=operator.itemgetter(1))

[('a', 1), ('b', 2), ('c', 3)]

In [38]:
d.keys()[d.values().index(max(d.values()))]

'c'